googleで検索して見つけた以下のページの行列の多項式への作用の定義を実装

https://ask.sagemath.org/question/60980/how-to-define-group-actions-on-sagemath/

In [33]:
from sage.categories.action import Action
from sage.groups.matrix_gps.matrix_group import is_MatrixGroup
from sage.rings.polynomial.multi_polynomial_ring import is_MPolynomialRing

class InverseLinearMapSubstitutionAction(Action):
    def __init__(self, G, S):
        if not is_MatrixGroup(G):
            raise TypeError("Not a matrix group: %s" % G)
        if not is_MPolynomialRing(S):
            raise TypeError("Not a multivariate polynomial ring: %s" % S)
        if not G.degree() == S.ngens():
            raise ValueError("Degree of matrix group (%d) does not match number of generators of polynomial ring (%d)" % (G.degree(), S.ngens()))
        if not S.base_ring().has_coerce_map_from(G.base_ring()):
            raise ValueError("Base rings not compatible")
        super().__init__(G, S, is_left=True, op=operator.mul)

    def _act_(self, g, f):
        return f(*(g.inverse() * vector(self.domain(), self.domain().gens())))

    def _repr_name_(self):
        return "inverse linear map substitution action"

剰余類の代表系を計算する関数を実装

In [34]:
from sage.groups.group import is_Group
import time

def cosetRep(G,H):
    # 時間計測開始
    time_sta = time.perf_counter()
    if not is_Group(G):
        raise TypeError("Not a group: %s" % G)
    if not is_Group(H):
        raise TypeError("Not a group: %s" % H)
    
    rep=[]
    g=set(G.list())
    h=set(G(e) for e in H.list())
    coset=set()
    while g:
        p=g.pop()
        rep.append(p)
        coset = set(p*e for e in h)
        g=g-coset
        coset.clear()
        if len(rep)*H.order()==G.order():
            break;
            
    # 時間計測終了
    time_end = time.perf_counter()
    # 経過時間（秒）
    tim = time_end- time_sta

#    print("%d 秒経過" % tim)
    #[結果]
    
    return rep

ルート系とWeyl group $W_G$（の置換群表現版の$S_G$）を生成　※剰余類の代表系計算の軽量化のため、置換群を利用

In [35]:
n=5
Delta_G=RootSystem(['A',n-1])
S_G=WeylGroup(Delta_G,implementation="permutation")
DynkinDiagram(Delta_G)

O---O---O---O
1   2   3   4   
A4

cross outするインデックスを設定

In [36]:
crossed_out_index=[2]
crossed_out_index.sort()

対応するcrossed Dynkin diagramを表示

In [37]:
CartanType(Delta_G).marked_nodes(crossed_out_index).dynkin_diagram()

O---X---O---O
1   2   3   4   
A4 with node 2 marked

simple reflectionを取得

In [38]:
s_G=S_G.simple_reflections()

uncrossed nodeに対応するsimple reflectionを取得

In [39]:
s_H=[ s_G[i] for i in set(range(1,len(s_G)+1))-set(crossed_out_index) ]

uncrossed nodeに対応するsimple reflectionから$W_G$の部分群$W_H$（の置換群表現版の$S_H$）を生成

In [40]:
S_H=S_G.subgroup(s_H)
print('W_Gの位数: %d'%S_G.order())
print('W_Hの位数: %d'%S_H.order())
print('代表系のサイズ: %d'%(S_G.order()/S_H.order()).round())

W_Gの位数: 120
W_Hの位数: 12
代表系のサイズ: 10


冒頭で実装した関数を用いて, $W_G/W_H$（の置換群版の$S_G/S_H$）の代表系を計算

作用させる際は行列としての作用となるため、本来のWeyl group $W_G$を用意し, 代表系を$W_G$の元に変換

In [41]:
W_G = WeylGroup(Delta_G)
W_G_mod_W_H = [W_G(w) for w in cosetRep(S_G,S_H)]

多項式環とWeyl group $W_G$ の多項式環への作用を生成

In [42]:
R = PolynomialRing(QQ, 'x', W_G.degree())
weyl_group_action = InverseLinearMapSubstitutionAction(W_G, R)

変数のリストを用意

In [43]:
x = R.gens(); x

(x0, x1, x2, x3, x4)

positive root $P_G$をambient spaceに用意する.

In [44]:
#ambient spaceを生成
L=Delta_G.ambient_space()

#positive rootをambient spaceの元に変換
P_G=[ L(x) for x in Delta_G.root_lattice().positive_roots() ];P_G

[(1, -1, 0, 0, 0),
 (0, 1, -1, 0, 0),
 (0, 0, 1, -1, 0),
 (0, 0, 0, 1, -1),
 (1, 0, -1, 0, 0),
 (0, 1, 0, -1, 0),
 (0, 0, 1, 0, -1),
 (1, 0, 0, -1, 0),
 (0, 1, 0, 0, -1),
 (1, 0, 0, 0, -1)]

uncrossed nodeから生成されるpositive rootの集合$P_H$をambient spaceに用意する.

In [45]:
# positive rootからcutoutされるsimple rootを引き, positiveでなければuncrossed nodeから生成されると判定
Pi_H=[ x for x in Delta_G.root_lattice().positive_roots()]
for i in crossed_out_index:
    Pi_H=[ x for x in Pi_H if not (x-(Delta_G.root_lattice().simple_roots())[i]).is_positive_root()]
    # ambient spaceの元に変換
P_H=[ L(x) for x in Pi_H ]; P_H

[(1, -1, 0, 0, 0), (0, 0, 1, -1, 0), (0, 0, 0, 1, -1), (0, 0, 1, 0, -1)]

In [46]:
# the weights of the tangenst spaces at torus-fixed points
flag_tangent_weights = set(P_G)-set(P_H)
flag_tangent_weights = [sum( r[l]*x[l] for l in range(len(x))) for r in flag_tangent_weights]

In [47]:
# denominator appearing in localization formula
# (the product of the above weights)
denominator_in_localization = prod(flag_tangent_weights)
# dimension of the flag variety
dim_flag = len(flag_tangent_weights)
t, k, l = var('t k l')
# Todd polynomial (truncated at dim_flag)
td(t) = taylor(t/(1-exp(-t)),t,0,dim_flag)
# truncated exponential function
truncated_exp = lambda x: sum( [x^i /factorial(i) for i in range(dim_flag+1)])

In [48]:
# homogeneous part
homogeneous_part = lambda F, degree: sum( c*m for c, m in F if m.total_degree() == degree )

In [49]:
# Chern character for the equivariant bundel designated by m times the fundamental weight of the crossed out index
m = 3
ch = truncated_exp(m*sum([x[k] for k in range(crossed_out_index[0])]))
ch_td = (ch*prod([td(l) for l in flag_tangent_weights])).polynomial(QQ)

Numerical computation

In [50]:
def numerical_integration_by_localization(f):
    random_x = [RealField(1000)(random()) for i in range(W_G.degree())]
    orbit_of_random_x = [(w.inverse()*vector(RealField(1000),random_x)).list() for w in W_G_mod_W_H]
    top_of_f = homogeneous_part(f, dim_flag)
    return sum([top_of_f(x)/denominator_in_localization(x) for x in orbit_of_random_x]).round()

In [51]:
numerical_integration_by_localization(ch_td)

175

Comparison with the exact result

In [52]:
# dimension of the irreducible representation of G
# with m times the fundamental weight of the crossed out index as the highest weight
L.weyl_dimension(m*L.fundamental_weights()[crossed_out_index[0]])

175

Symbolic computation

In [53]:
symbolic_integration_by_localization = lambda f: sum(
    [ weyl_group_action(w, f)/weyl_group_action(w, denominator_in_localization) for w in W_G_mod_W_H]
)

In [54]:
symbolic_integration_by_localization(homogeneous_part(ch_td, dim_flag))

175

Chern classes

In [55]:
total_chern = prod(1+x for x in flag_tangent_weights)
chern_classes = [ homogeneous_part(total_chern, i) for i in (0..dim_flag) ]

Numerical computation of Chern numbers

In [56]:
def chern_number(deg_exp: dict) -> int:
    if sum(d*e for (d,e) in deg_exp.items()) != dim_flag:
        return 0
    else:
        return numerical_integration_by_localization(
            prod([chern_classes[d]^e for (d,e) in deg_exp.items()])
        )

The degree of the flag variety

In [57]:
chern_number({1:dim_flag})

78125

HHR by chern classes

- constants

In [58]:
dim_M = dim_flag
rank_E = 1

- The ring generated by chern classes with their degrees
  (Degrees in SageMath must be positive, so we treat c_0 with replacing by numbers.)

In [59]:
ring_for_ch = PolynomialRing(QQ, [ f'c{i}_E' for i in (1..rank_E) ], order=TermOrder('wdeglex', tuple(1..rank_E)))
ring_for_ch

Univariate Polynomial Ring in c1_E over Rational Field

In [60]:
ring_for_td = PolynomialRing(QQ, [ f'c{i}_M' for i in (1..dim_M) ], order=TermOrder('wdeglex', tuple(1..dim_M)))
ring_for_td

Multivariate Polynomial Ring in c1_M, c2_M, c3_M, c4_M, c5_M, c6_M over Rational Field

- The list of degreewise chern characters

In [61]:
import re
singular.lib('chern.lib')
r = singular.ring(0, f'(c(1..{rank_E}))', 'dp')
l = singular.list(f'c(1..{rank_E})')
ch_str_list = singular.chAll(l, dim_M).sage_structured_str_list()
chern_character = [ ring_for_ch(rank_E) ] + [ ring_for_ch(re.sub('c\(([0-9]+)\)', 'c\\1_E', s)) for s in ch_str_list ]
chern_character

[1, c1_E, 1/2*c1_E^2, 1/6*c1_E^3, 1/24*c1_E^4, 1/120*c1_E^5, 1/720*c1_E^6]

- The list of degreewise todd classes

In [62]:
r = singular.ring(0, f'(c(1..{dim_M}))', 'dp')
l = singular.list(f'c(1..{dim_M})')
todd_str_list = singular.todd(l).sage_structured_str_list()
todd_classes = [ ring_for_td(1) ] + [ ring_for_td(re.sub('c\(([0-9]+)\)', 'c\\1_M', s)) for s in todd_str_list ]
todd_classes

[1,
 1/2*c1_M,
 1/12*c1_M^2 + 1/12*c2_M,
 1/24*c1_M*c2_M,
 -1/720*c1_M^4 + 1/180*c1_M^2*c2_M + 1/720*c1_M*c3_M + 1/240*c2_M^2 - 1/720*c4_M,
 -1/1440*c1_M^3*c2_M + 1/1440*c1_M^2*c3_M + 1/480*c1_M*c2_M^2 - 1/1440*c1_M*c4_M,
 1/30240*c1_M^6 - 1/5040*c1_M^4*c2_M + 1/12096*c1_M^3*c3_M + 11/60480*c1_M^2*c2_M^2 - 1/12096*c1_M^2*c4_M + 11/60480*c1_M*c2_M*c3_M - 1/30240*c1_M*c5_M + 1/6048*c2_M^3 - 1/6720*c2_M*c4_M - 1/60480*c3_M^2 + 1/30240*c6_M]

- The HRR formula

In [63]:
hrr = lambda chern_classes_of_E, chern_classes_of_M: numerical_integration_by_localization( 
        sum(
            chern_character[i](chern_classes_of_E)
            * todd_classes[dim_M - i](chern_classes_of_M)
        for i in (0..dim_M))
    )


- Test

In [64]:
chern_classes_of_E = [ m*sum([x[k] for k in range(crossed_out_index[0])])]
chern_classes_of_M = [ 
    homogeneous_part(prod(1+x for x in flag_tangent_weights), i)
    for i in (1..dim_flag) 
]

Comparison of chern chracters

In [65]:
[ chern_character[i](chern_classes_of_E) for i in (0..dim_flag) ]

[1,
 3*x0 + 3*x1,
 9/2*x0^2 + 9*x0*x1 + 9/2*x1^2,
 9/2*x0^3 + 27/2*x0^2*x1 + 27/2*x0*x1^2 + 9/2*x1^3,
 27/8*x0^4 + 27/2*x0^3*x1 + 81/4*x0^2*x1^2 + 27/2*x0*x1^3 + 27/8*x1^4,
 81/40*x0^5 + 81/8*x0^4*x1 + 81/4*x0^3*x1^2 + 81/4*x0^2*x1^3 + 81/8*x0*x1^4 + 81/40*x1^5,
 81/80*x0^6 + 243/40*x0^5*x1 + 243/16*x0^4*x1^2 + 81/4*x0^3*x1^3 + 243/16*x0^2*x1^4 + 243/40*x0*x1^5 + 81/80*x1^6]

In [66]:
[ homogeneous_part(ch, i) for i in (0..dim_flag) ] == [ chern_character[i](chern_classes_of_E) for i in (0..dim_flag) ]

True

Comparison of todd classes

In [67]:
[ todd_classes[i](chern_classes_of_M) for i in (0..dim_flag)]

[1,
 3/2*x0 + 3/2*x1 - x2 - x3 - x4,
 x0^2 + 9/4*x0*x1 + x1^2 - 17/12*x0*x2 - 17/12*x1*x2 + 5/12*x2^2 - 17/12*x0*x3 - 17/12*x1*x3 + x2*x3 + 5/12*x3^2 - 17/12*x0*x4 - 17/12*x1*x4 + x2*x4 + x3*x4 + 5/12*x4^2,
 3/8*x0^3 + 3/2*x0^2*x1 + 3/2*x0*x1^2 + 3/8*x1^3 - 7/8*x0^2*x2 - 2*x0*x1*x2 - 7/8*x1^2*x2 + 13/24*x0*x2^2 + 13/24*x1*x2^2 - 1/12*x2^3 - 7/8*x0^2*x3 - 2*x0*x1*x3 - 7/8*x1^2*x3 + 4/3*x0*x2*x3 + 4/3*x1*x2*x3 - 5/12*x2^2*x3 + 13/24*x0*x3^2 + 13/24*x1*x3^2 - 5/12*x2*x3^2 - 1/12*x3^3 - 7/8*x0^2*x4 - 2*x0*x1*x4 - 7/8*x1^2*x4 + 4/3*x0*x2*x4 + 4/3*x1*x2*x4 - 5/12*x2^2*x4 + 4/3*x0*x3*x4 + 4/3*x1*x3*x4 - x2*x3*x4 - 5/12*x3^2*x4 + 13/24*x0*x4^2 + 13/24*x1*x4^2 - 5/12*x2*x4^2 - 5/12*x3*x4^2 - 1/12*x4^3,
 19/240*x0^4 + 9/16*x0^3*x1 + x0^2*x1^2 + 9/16*x0*x1^3 + 19/240*x1^4 - 211/720*x0^3*x2 - 59/48*x0^2*x1*x2 - 59/48*x0*x1^2*x2 - 211/720*x1^3*x2 + 107/360*x0^2*x2^2 + 25/36*x0*x1*x2^2 + 107/360*x1^2*x2^2 - 11/120*x0*x2^3 - 11/120*x1*x2^3 + 1/240*x2^4 - 211/720*x0^3*x3 - 59/48*x0^2*x1*x3 - 59/48*x0*

In [68]:
[ homogeneous_part(prod([td(l) for l in flag_tangent_weights]).polynomial(QQ), i) for i in (0..dim_flag) ] == [ todd_classes[i](chern_classes_of_M) for i in (0..dim_flag)]

True

Comparison of top term of ch_td

In [69]:
top_of_ch_td = sum (chern_character[i](chern_classes_of_E) * todd_classes[dim_M - i](chern_classes_of_M)
    for i in (0..dim_M)
)
homogeneous_part(ch_td, dim_flag) == top_of_ch_td

True

HRR

In [70]:
hrr(chern_classes_of_E, chern_classes_of_M)

175

In [71]:
xs = [
    numerical_integration_by_localization(
        chern_character[i](chern_classes_of_E)
        * todd_classes[dim_M - i](chern_classes_of_M)
    )
    for i in (0..dim_M)
];xs

[1, 9, 30, 53, 51, 25, 5]

In [72]:
sum(xs)

174